```
Flores Aquino Luis Enrique
171564
```

# DEPENDENCIAS

In [38]:
#Instalacion
#Llamado
import numpy as np
import itertools as it

# DEFINICION DE APRIORI

## Metodos aledaños

Metodos que ayudan a la ejecución del algoritmo APRIORI

In [64]:
def ContarItems(playlists):
    diccionario_frecuencias = {}
    #Contabilizamos las aparicioesn
    #Y guardamos el nro de playlist en la que aparecen
    for k in range(len(playlists)):
        for j in range(len(playlists[k])):
            palabra=playlists[k][j]
            if palabra in diccionario_frecuencias:
                diccionario_frecuencias[palabra][0] += 1
                diccionario_frecuencias[palabra][1].add(k)
            else:
                diccionario_frecuencias[palabra] = [1,{k}]
    itemset = diccionario_frecuencias.keys()
    valores = diccionario_frecuencias.values()
    cantidad = []
    posiciones = []
    #Separamos la cantidad de las posciones
    for val in valores:
        cantidad.append(val[0])
        posiciones.append(val[1])
    return list(itemset),cantidad,posiciones
    
def GenerarYContarNuevosItemset(Old_Itemset,Old_Posiciones):
    PosItemSet = []
    Valores = []
    Posiciones = []
    for k in range(0,len(Old_Itemset)-1):
        for j in range(k+1,len(Old_Itemset)):
            #Realizamos la operacion A U B sobre 2 items antiguos
            Posible_Item = sorted(list(set(Old_Itemset[k]+Old_Itemset[j])))
            #Si el item resultante solo tiene 1 elemeno más que sus padres
            #Se le considera como posible itemset
            if(len(Posible_Item)==len(Old_Itemset[0])+1 
               and Posible_Item not in PosItemSet):
            #Contamos la cantidad de apariciones que posee
            #El nuevo itemset en base a un match de las posiciones
            #De sus padres
                PosicionesItemset = Old_Posiciones[k]&Old_Posiciones[j]
                Valores.append(len(list(PosicionesItemset)))
                Posiciones.append(PosicionesItemset)
                PosItemSet.append(Posible_Item)
    return PosItemSet,Valores,Posiciones

def GenerarPosibleItemset(Old_Itemset):
    PosItemSet = []
    for k in range(0,len(Old_Itemset)-1):
        for j in range(k+1,len(Old_Itemset)):
            #Realizamos la operacion A U B sobre 2 items antiguos
            Posible_Item = sorted(list(set(Old_Itemset[k]+Old_Itemset[j])))
            #Si el item resultante solo tiene 1 elemeno más que sus padres
            #Se le considera como posible itemset
            if(len(Posible_Item)==len(Old_Itemset[0])+1 
               and Posible_Item not in PosItemSet):
                PosItemSet.append(Posible_Item)
    return PosItemSet

def ContarAparicionItemSet(Pos_Itemset, Dataset):
    #Generamos un vector de igual longitud
    #Al posible item set para ir acumulando sus apareciones
    Contador_Tot = [0]*len(Pos_Itemset)
    for k in range(len(Pos_Itemset)):
        
        Item = Pos_Itemset[k]
        #Recorremos el dataset para contar las veces que contiene
            #el item actual
        for tupla in Dataset:
            #Hacemos la operación Item U Tupla
            tupla = set(tupla)
            temp = list(set(Item).union(tupla))
            tupla = list(tupla)
            #Luego comprobamos si cantidad de elementos
            #Si es igual a la de la tupla, en caso sea verdad
            #el item se encuentra dentro de la tupla
            if(len(temp)==len(tupla)):
                Contador_Tot[k]+=1
    return Contador_Tot


def LimpiarItemset(Itemset,Valores,Posiciones,umbral):
    #Funcion que se encarga de eliminar los valores del itemset
    #Cuya confianza no supere el umbral dado
    items_validos = []
    cantVal = 0
    for valor_act in Valores:
        items_validos.append(valor_act>=umbral)
        if(valor_act>=umbral):
            cantVal+=1
    Itemset_limpio = list(it.compress(Itemset,items_validos))
    Valores_limpio = list(it.compress(Valores,items_validos))
    Posiciones_limpio = list(it.compress(Posiciones,items_validos))
    return Itemset_limpio,Valores_limpio,Posiciones_limpio



#Obtenemos todas las permutaciones posibles
def ObtenerPosiblesReglas(item):
    combinaciones = list(it.permutations(item))
    Xs_actuales = []
    Ys_actuales = []
    for tupla in combinaciones:
        tupla = list(tupla)
        #print(tupla)
        #Obtenemos todas las divisiones posibles
        #de cada colección de items
        for k in range(1,len(tupla)):
            #Basandonos en que las reglas son
            # X -> Y
            # los separamos para en 2 variables
            X, Y = tupla[:k],tupla[k:]

            # Ordenamos cada sub conjunto
            X.sort() 
            Y.sort()

            #La agregamos al total
            Xs_actuales.append(X)
            Ys_actuales.append(Y)
    return Xs_actuales,Ys_actuales


def LimpiarReglasRepetidas(X,Y):
    #Eliminamos las combinaciones iguales 
    validos = [True]*len(X)
    for k in range(len(X)-1):
        #Si el valor actual todavía es válido
        #Comprobamos si tiene algún otro repetido
        if(validos[k]):
            #Comprobamos si entre los de adelante
            #hay una igual a la regla actual
            for j in range(k+1,len(X)):
                #Si el elemento j todavía es valido
                #con X y Y iguales, se marca como
                # NO VALIDO
                if(validos[j] and 
                    X[k] == X[j] and
                    Y[k] == Y[j]):
                    validos[j]=False
    #Eliminamos los no validos, tanto para X como Y
    X = list(it.compress(X,validos))
    Y = list(it.compress(Y,validos))
    return X,Y

def MostrarReglas(Xs,Ys):
    for k in range(len(Xs)):
        print(f'{Xs[k]} -> {Ys[k]}')

## Obtener la frecuencia del itemset
En base a un umbral mínimo

In [65]:
def get_frequent_itemsets(playlists, min_support=0.2):
    #Creamos 2 arreglos
        #Uno almacenará todos los itemsets
        #El otro su soporte
    All_Itemsets = []
    All_values = []
    num_iter=1

    #Ordenamos las playlists
    #for playlist in playlists:
    #    playlist.sort()
    
    print('Calculando itemsets de long %2i'%(num_iter))
    
    #Obtenemos todos los itemsets de 1
    itemset,val_item,last_Posiciones = ContarItems(playlists)
    #PRINT DE MUESTRA
    #for k in range(len(itemset)):
    #    print(itemset[k] + '|' + str(val_item[k]))

    #Normalizamos en valores de 0 a 1 los valores acumulados que se tienen
    long_playlists = len(playlists)
    val_item= [x/long_playlists for x in val_item]
    
    
    #Eliminamos todos los objetos que no pasan el umbra "min_support"
    valid_itemset,valid_values, last_Posiciones= LimpiarItemset(itemset,val_item,last_Posiciones,min_support)

    #Ordenamos los valores
    temp_duplas = dict(zip(valid_itemset,valid_values)).items()
    temp_duplas2 = dict(zip(valid_itemset,last_Posiciones)).items()
    valid_itemset = []
    valid_values = []
    valid_Posiciones = []
    for dupla in sorted(temp_duplas):
        valid_itemset.append(dupla[0])
        valid_values.append(dupla[1])
    for dupla in sorted(temp_duplas2):
        valid_Posiciones.append(dupla[1])
    
    #PRINT DE MUESTRA
    #for k in range(len(valid_itemset)):
    #   print(valid_itemset[k] + '|' + str(valid_values[k]))

    #PROCEDEMOS A INICIAR UN CICLO PARA CALCULAR ITEMSETS DE MAYOR CARDINALIDAD
    last_itemset = [[x] for x in valid_itemset]
    
    #PRINT DE MUESTRA
    print("└──Terminado || cant. elementos (%i)"%(len(last_itemset)))
    while(True):
        
        #Comprobamos si nuestra última iteración tiene todavía items válidos
        if(len(last_itemset)==0):
            break

        #Añadimos nuestra última iteracion al total de itemsets
        All_Itemsets.append(last_itemset)
        All_values.append(valid_values)

        #Si solo tenemos un item en el itemset tambien acabamos todo
        if(len(last_itemset)==1):
            break
        
        num_iter+=1
        print('Calculando itemsets de long %2i'%(num_iter))

        #Creamos todas las tuplas posibles de la próxima iteración
        #Y su cantidad de apariciones
        Pos_Itemset,Val_PosItems,valid_Posiciones = GenerarYContarNuevosItemset(last_itemset,valid_Posiciones)

        #Contamos las apariciones del posible item set a lo largo
            #de todo el data set
        #Val_PosItems = ContarAparicionItemSet(Pos_Itemset,playlists)
                  
        #Normalizamos en valores de 0 a 1 los valores acumulados que se tienen

        Val_PosItems= [x/long_playlists for x in Val_PosItems]

        #Eliminamos los valores que no pasan el umbral
        last_itemset, valid_values,last_Posiciones = LimpiarItemset(Pos_Itemset,Val_PosItems,last_Posiciones,min_support)
        #PRINT DE MUESTRA
        #for k in range(len(Pos_Itemset)):
        #    print(str(Pos_Itemset[k])+'|'+ str(Val_PosItems[k]))
        print("└──Terminado || cant. elementos (%i)"%(len(last_itemset)))
    return All_Itemsets,All_values

## Generar las reglas de asociacion

In [66]:
def generate_association_rules(frequent_itemsets, confidence = 0, lift =0):
    All_rules = []
    All_lift = []
    All_confidence = []
    All_support = []
    Long_regla_actual = 2
    itemsets = frequent_itemsets[0]
    itemsvalues = frequent_itemsets[1]
    Xs = []
    Ys = []
    print('Calculas todas las posibles reglas')
    while(Long_regla_actual<=len(itemsets)):
        #Obtenemos todas las posibles reglas
        itemset_actual = itemsets[Long_regla_actual-1]
        #Obtenemos las posibles regla de cada item
        for item in itemset_actual:
            xs,ys = ObtenerPosiblesReglas(item)
            Xs+=xs
            Ys+=ys
        Long_regla_actual+=1
    print('Limpieza de reglas repetidas')
    #Una vez con todas las posibles reglas
    #De la forma X->Y
    #Limpiamos las reglas repetidas
    Xs,Ys = LimpiarReglasRepetidas(Xs,Ys)
    

    #CALCULO DEL LIFT Y CONFIANZA
    #Calculo del lift
    
    #Recorremos las reglas y calculamos su lift
    #y confianza, además de verificar que pasen
    #Los umbrales
    print('Calculando lift y confianza')
    for k in range(len(Xs)):
        x = Xs[k]
        y = Ys[k]
        xuy = x+y
        xuy.sort()
        #Obtenemos los valores necesarios para el calculo
        pos = itemsets[len(x)-1].index(x)
        Px = itemsvalues[len(x)-1][pos]
        pos = itemsets[len(y)-1].index(y)
        Py = itemsvalues[len(y)-1][pos]
        pos = itemsets[len(xuy)-1].index(xuy)
        Pxuy = itemsvalues[len(xuy)-1][pos]
        #Calculamos la confianza y el lift
        confianza_act = Pxuy/Px
        lift_act = Pxuy/Px/Py
        #comprobamos si pasa el umbral
        if(lift_act >= lift and confianza_act >= confidence):
            All_rules.append([x,y])
            All_confidence.append(confianza_act)
            All_lift.append(lift_act)
            All_support.append(Pxuy)
    print('FIN')
    return  All_rules,All_confidence,All_lift,All_support

# EJECUCION

## LECTURA DE DATOS

In [67]:
data_spotify = np.load('spotify.npy',allow_pickle=True)

## TRATAMIENTO

In [68]:
X = data_spotify.tolist()
spotify_playlists = [X[k] for k in X]

# Ejecucion

## Obtencion de los itemset

In [69]:
All_Itemsets,All_values = get_frequent_itemsets(spotify_playlists, 0.015)
#Completo 12m 33.4s || 0.04
#         29m  1.6s || 0.02
#5000      4m 27.8s || 0.02

Calculando itemsets de long  1
└──Terminado || cant. elementos (371)
Calculando itemsets de long  2
└──Terminado || cant. elementos (22)
Calculando itemsets de long  3
└──Terminado || cant. elementos (0)


### Mostrar itemsets adquiridos

In [70]:
print(f'Longitud máxima encontrada: {len(All_Itemsets)}')
# MOSTRAR ITEMSETS VALIDOS ENCONTRADOS
for k in range(len(All_Itemsets)):
    print('===== %3i ======='%(k+1))
    for j in range(len(All_Itemsets[k])):
        print('%s || %3.2f'%(All_Itemsets[k][j],All_values[k][j]*100))

Longitud máxima encontrada: 2
=====   1 =======
["'Till I Collapse"] || 1.72
['0 To 100 / The Catch Up'] || 1.63
['1-800-273-8255'] || 2.64
['2 Phones'] || 1.51
['24K Magic'] || 1.68
['679 (feat. Remy Boyz)'] || 2.46
['7 Years'] || 1.97
['7/11'] || 1.52
['A Thousand Years'] || 1.53
['Africa'] || 1.71
['Again'] || 2.14
["Ain't No Mountain High Enough"] || 2.33
['Alive'] || 2.33
['All I Want'] || 2.34
['All My Friends'] || 1.88
['All Night'] || 1.58
['All Night (feat. Knox Fortune)'] || 1.66
['All Star'] || 1.68
['All The Small Things'] || 1.65
['All Time Low'] || 1.92
['All of Me'] || 2.41
['Alone'] || 2.15
['Alright'] || 2.81
['Angel'] || 2.25
['Antidote'] || 2.64
['Attention'] || 1.56
['Back To Back'] || 1.67
['Bad and Boujee (feat. Lil Uzi Vert)'] || 3.53
['Bang Bang'] || 1.87
['Bank Account'] || 2.31
['Beautiful'] || 2.02
['Beautiful Girls'] || 1.54
['Believe'] || 2.20
['Believer'] || 2.54
['Best I Ever Had'] || 1.57
['Better Together'] || 1.58
['Billie Jean'] || 1.99
['Black Beatle

## Calcular reglas de asociacion

In [71]:
Reglas,Confianza,Lift,Soporte = generate_association_rules([All_Itemsets,All_values],0.50,2.75)

Calculas todas las posibles reglas
Limpieza de reglas repetidas
Calculando lift y confianza
FIN


### Mostrar reglas de asociacion

In [72]:
for k in range(len(Reglas)):
    print(f"{k+1} ======")
    print(f'{Reglas[k][0]} -> {Reglas[k][1]}')
    print(f'soporte: {Soporte[k]*100}')
    print(f'confianza: {Confianza[k]*100}')
    print(f'lift: {Lift[k]}')

1 ======
['Bounce Back'] -> ['Bad and Boujee (feat. Lil Uzi Vert)']
soporte: 1.69
confianza: 56.333333333333336
lift: 15.958451369216244
2 ======
['Bounce Back'] -> ['HUMBLE.']
soporte: 1.5599999999999998
confianza: 52.0
lift: 11.063829787234043
3 ======
['Congratulations'] -> ['HUMBLE.']
soporte: 2.1399999999999997
confianza: 52.195121951219505
lift: 11.10534509600415
4 ======
['Mask Off'] -> ['Congratulations']
soporte: 1.6199999999999999
confianza: 51.26582278481012
lift: 12.503859215807346
5 ======
['XO TOUR Llif3'] -> ['Congratulations']
soporte: 1.79
confianza: 54.242424242424235
lift: 13.229859571322985
6 ======
['goosebumps'] -> ['Congratulations']
soporte: 1.55
confianza: 50.324675324675326
lift: 12.27431105479886
7 ======
['DNA.'] -> ['HUMBLE.']
soporte: 1.9
confianza: 81.89655172413794
lift: 17.424798239178283
8 ======
['Mask Off'] -> ['HUMBLE.']
soporte: 2.04
confianza: 64.55696202531645
lift: 13.735523835173714
9 ======
['XO TOUR Llif3'] -> ['HUMBLE.']
soporte: 2.04
confia

## INTERPTRETACIÓN DE LAS REGLAS OBTENIDAS
Escogiendo las 4 reglas con mayor LIFT:

```
DNA -> HUMBLE
soporte: 1.9
confianza: 81.90
lift: 17.42


Bounce Back -> Bad and Boujee (feat. Lil Uzi Vert)
soporte: 1.69
confianza: 56.33
lift: 15.96


Mask Off -> XO TOUR Llif3
soporte: 1.63
confianza: 51.58
lift: 15.63


Mask Off -> HUMBLE
soporte: 2.04
confianza: 64.56
lift: 13.74
```

De las 4 canciones que pertenecen a las reglas:

|     | DNA | Humble | Mask Off | XO TOUR Llif3 | Bounce Back | Bad and Boujee |
| --- | --- | --- | --- | --- | --- | --- |
| <b>Artista | Kendrick Lamar |  Kendrick Lamar | Future | Lil Uzi Vert | Big Sean | Migos |
| <b>Año |  2017 |  2017 | 2017 | 2017 |  | 2016 | 2017 | 2017 |
| <b>Genero |  Hip-hop/trap | Hip-hop/rap | Hip-hop/trap | Hip-hop/rap | Hip-hop/rap | Hip-hop/trap|

+ Vemos que las 6 pertenecen al género Hip-Hop
+ Todas las conciones son del año 2017, a excepción de una, lo cual nos puede dar a conocer la fecha con la que se hizo el data set
+ Todos los artistas son hombres y ninguno pasa de los 40 años 
+ Todos ellos tienen al menos una colaboración con otro de los artistas mencionados
+ No podemos extraer más data debido a que no poseemos la metadata del dataset como edad o género de los dueños de las playlist
+ La regla DNA -> HUMBLE, es la de mayor lift y esto puede ser a que son canciones de un mismo artista y que son del mismo disco 'DAMN'

